In [16]:
import boto3

# Crear cliente de EC2
ec2_client = boto3.client('ec2', region_name='us-east-1')

# Listar instancias EC2
try:
    response = ec2_client.describe_instances()
    instances = response.get('Reservations', [])
    print("Instancias EC2:")
    for reservation in instances:
        for instance in reservation.get('Instances', []):
            print(f"- ID: {instance['InstanceId']}, Estado: {instance['State']['Name']}")
except Exception as e:
    print("Error al listar instancias EC2:", str(e))

# Crear cliente de S3
s3_client = boto3.client('s3', region_name='us-east-1')

# Listar buckets S3
try:
    response = s3_client.list_buckets()
    buckets = response.get('Buckets', [])
    print("Buckets S3:")
    for bucket in buckets:
        print(f"- {bucket['Name']}")
except Exception as e:
    print("Error al listar buckets S3:", str(e))

Instancias EC2:
- ID: i-02957a487a3ce9d30, Estado: terminated
Buckets S3:
- prueba010206


## Crear una instancia EC2, ejecutarla, pararla y eliminarla

In [17]:
# Crear una instancia EC2
try:
    ec2_response = ec2_client.run_instances(
        ImageId='ami-053a45fff0a704a47',  # Reemplaza con el ID de la AMI que desees usar
        InstanceType='t2.micro',  # Tipo de instancia t2.micro
        KeyName='prueba2',  # Nombre de la llave para acceder a la instancia(Creado otro dia en clase)
        MinCount=1,
        MaxCount=1,
        Placement={
            'AvailabilityZone': 'us-east-1d'  # Zona de disponibilidad
        },
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': 'tarea_almacenamiento'
                    }
                ]
            }
        ]
    )
    instance_id = ec2_response['Instances'][0]['InstanceId']
    print(f"Instancia EC2 creada con ID: {instance_id}")
except Exception as e:
    print("Error al crear la instancia EC2:", str(e))

Instancia EC2 creada con ID: i-01a8491f792ec6dc3


In [18]:
# Ejecutar la instancia EC2
try:
    ec2_client.start_instances(InstanceIds=[instance_id])
    print(f"Instancia EC2 {instance_id} iniciada")
except Exception as e:
    print("Error al iniciar la instancia EC2:", str(e))

Instancia EC2 i-01a8491f792ec6dc3 iniciada


In [14]:
# Parar la instancia EC2
try:
    ec2_client.stop_instances(InstanceIds=[instance_id])
    print(f"Instancia EC2 {instance_id} parada")
except Exception as e:
    print("Error al parar la instancia EC2:", str(e))

Instancia EC2 i-02957a487a3ce9d30 parada


In [15]:
# Eliminar la instancia EC2
try:
    ec2_client.terminate_instances(InstanceIds=[instance_id])
    print(f"Instancia EC2 {instance_id} eliminada")
except Exception as e:
    print("Error al eliminar la instancia EC2:", str(e))

Instancia EC2 i-02957a487a3ce9d30 eliminada


## Crear un EBS y asociarlo al EC2 anterior y añadir un archivo

In [6]:
import time
import paramiko

# Crear un volumen EBS
try:
    volume_response = ec2_client.create_volume(
        AvailabilityZone='us-east-1d',  # Zona de disponibilidad de la instancia EC2
        Size=1,  # Tamaño del volumen en GB
        VolumeType='gp2',  # Tipo de volumen
        TagSpecifications=[
            {
                'ResourceType': 'volume',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': 'tarea_almacenamiento_ebs'
                        },
                        {
                            'Key': 'AvailabilityZone',
                            'Value': 'us-east-1d'
                    }
                ]
            }
        ]
    )
    volume_id = volume_response['VolumeId']
    print(f"Volumen EBS creado con ID: {volume_id}")
except Exception as e:
    print("Error al crear el volumen EBS:", str(e))

# Esperar a que el volumen esté disponible
while True:
    volume_status = ec2_client.describe_volumes(VolumeIds=[volume_id])
    if volume_status['Volumes'][0]['State'] == 'available':
        break
    time.sleep(5)

Volumen EBS creado con ID: vol-0f4fa8c2a28bc5eaf


In [19]:
# Asociar el volumen EBS a la instancia EC2
try:
    ec2_client.attach_volume(
        VolumeId=volume_id,
        InstanceId=instance_id,
        Device='/dev/sdf'  # Dispositivo en el que se montará el volumen
    )
    print(f"Volumen EBS {volume_id} asociado a la instancia EC2 {instance_id}")
except Exception as e:
    print("Error al asociar el volumen EBS a la instancia EC2:", str(e))

Volumen EBS vol-0f4fa8c2a28bc5eaf asociado a la instancia EC2 i-01a8491f792ec6dc3


In [20]:
# Configurar la conexión SSH
key = paramiko.RSAKey(filename='prueba2.pem')  # Reemplaza con la ruta a tu archivo de clave privada
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    ssh_client.connect(hostname='ec2-3-89-110-72.compute-1.amazonaws.com', username='ec2-user', pkey=key)  # Reemplaza con la dirección pública de tu instancia EC2

    # Montar el volumen EBS
    commands = [
        'sudo mkfs -t ext4 /dev/xvdf',  # Formatear el volumen
        'sudo mkdir /mnt/ebs',  # Crear un directorio para montar el volumen
        'sudo mount /dev/xvdf /mnt/ebs',  # Montar el volumen
        'echo "Hello, EBS!" | sudo tee /mnt/ebs/hello.txt'  # Crear un archivo en el volumen montado
    ]
    for command in commands:
        stdin, stdout, stderr = ssh_client.exec_command(command)
        print(stdout.read().decode())
        print(stderr.read().decode())

    print("Archivo creado en el volumen EBS montado")
except Exception as e:
    print("Error al conectar a la instancia EC2 y montar el volumen EBS:", str(e))
finally:
    ssh_client.close()

Creating filesystem with 262144 4k blocks and 65536 inodes
Filesystem UUID: 3777ff68-556f-45c2-9340-eeb5b3e985bc
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376

Allocating group tables: 0/  done                            
Writing inode tables: 0/  done                            
Creating journal (8192 blocks): done
Writing superblocks and filesystem accounting information: 0/  done


mke2fs 1.46.5 (30-Dec-2021)





Hello, EBS!


Archivo creado en el volumen EBS montado


### Para conectarse y verificar que todo esta correctamente necesitamos darle una serie de permisos a prueba2.pem y entrar por ssh
- ssh -i "Key" ec2-user@ec2"ip publica".compute-1.amazonaws.com

## Crear un EFS, montarlo y añadir un archivo

In [23]:
# Crear un sistema de archivos EFS
efs_client = boto3.client('efs', region_name='us-east-1')

try:
    efs_response = efs_client.create_file_system(
        CreationToken='tarea_almacenamiento_efs',
        PerformanceMode='generalPurpose',
        Encrypted=False,
        Tags=[
            {
                'Key': 'Name',
                'Value': 'tarea_almacenamiento_efs'
            }
        ]
    )
    file_system_id = efs_response['FileSystemId']
    print(f"Sistema de archivos EFS creado con ID: {file_system_id}")
except Exception as e:
    print("Error al crear el sistema de archivos EFS:", str(e))

# Esperar a que el sistema de archivos esté disponible
while True:
    efs_status = efs_client.describe_file_systems(FileSystemId=file_system_id)
    if efs_status['FileSystems'][0]['LifeCycleState'] == 'available':
        break
    time.sleep(5)

Sistema de archivos EFS creado con ID: fs-07ca1385a36c1d82f


In [24]:
# Crear un punto de montaje para el EFS
try:
    mount_target_response = efs_client.create_mount_target(
        FileSystemId=file_system_id,
        SubnetId='subnet-0acefd304dedd5b8e',  # Reemplaza con el ID de tu subred
        SecurityGroups=['sg-0a1c2fd3b0eead5d9']  # Reemplaza con el ID de tu grupo de seguridad
    )
    mount_target_id = mount_target_response['MountTargetId']
    print(f"Punto de montaje creado con ID: {mount_target_id}")
except Exception as e:
    print("Error al crear el punto de montaje EFS:", str(e))

# Esperar a que el punto de montaje esté disponible
while True:
    mount_target_status = efs_client.describe_mount_targets(MountTargetId=mount_target_id)
    if mount_target_status['MountTargets'][0]['LifeCycleState'] == 'available':
        break
    time.sleep(5)

Punto de montaje creado con ID: fsmt-04b922b3551b40428


In [27]:
# Montar el EFS en la instancia EC2
try:
    ssh_client.connect(hostname='ec2-3-89-110-72.compute-1.amazonaws.com', username='ec2-user', pkey=key)  # Reemplaza con la dirección pública de tu instancia EC2

    # Montar el EFS
    commands = [
        'sudo yum install -y amazon-efs-utils',
        'sudo mkdir -p /mnt/efs',  # Crear el directorio si no existe
        f'sudo mount -t efs {file_system_id}:/ /mnt/efs',
        'echo "Hello, EFS!" | sudo tee /mnt/efs/hello.txt'  # Crear un archivo en el EFS montado
    ]
    for command in commands:
        stdin, stdout, stderr = ssh_client.exec_command(command)
        print(stdout.read().decode())
        print(stderr.read().decode())

    print("Archivo creado en el sistema de archivos EFS montado")
except Exception as e:
    print("Error al conectar a la instancia EC2 y montar el sistema de archivos EFS:", str(e))
finally:
    ssh_client.close()

Last metadata expiration check: 0:06:39 ago on Sat Mar 15 09:34:46 2025.
Package amazon-efs-utils-2.1.0-1.amzn2023.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!





Mount attempt 1/3 failed due to timeout after 15 sec, wait 0 sec before next attempt.
Mount attempt 2/3 failed due to timeout after 15 sec, wait 0 sec before next attempt.
b'mount.nfs4: Connection timed out'

Hello, EFS!


Archivo creado en el sistema de archivos EFS montado


## Crear un S3 Estándar, crear un cubo y añadir varias carpetas con un objeto que sea un archivo csv con varios datos para trabajar con él a posteriori y obtener le objeto

In [29]:
import os
import boto3

# Crear cliente S3
s3_client = boto3.client('s3')

# Nombre del bucket
bucket_name = 'bucket-estandar'  # Cambia el nombre para que sea único

# Crear el bucket en us-east-1 (sin LocationConstraint)
try:
    s3_client.create_bucket(Bucket=bucket_name)
    print(f"Bucket {bucket_name} creado exitosamente")
except Exception as e:
    print("Error al crear el bucket S3:", str(e))

# Añadir varias carpetas y subir un archivo CSV
file_path = '/home/feliciano/SBD/Boto3/best-selling-books.csv'
folders = ['carpeta1', 'carpeta2', 'carpeta3']
file_key = '/'.join(folders) + '/' + os.path.basename(file_path)

try:
    s3_client.upload_file(file_path, bucket_name, file_key)
    print(f"Archivo {file_path} subido exitosamente a {file_key} en el bucket {bucket_name}")
except Exception as e:
    print("Error al subir el archivo CSV:", str(e))

# Obtener el objeto subido
try:
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    print(f"Objeto obtenido exitosamente: {response['Body'].read().decode('utf-8')}")
except Exception as e:
    print("Error al obtener el objeto del bucket S3:", str(e))

Bucket bucket-estandar creado exitosamente
Archivo /home/feliciano/SBD/Boto3/best-selling-books.csv subido exitosamente a carpeta1/carpeta2/carpeta3/best-selling-books.csv en el bucket bucket-estandar
Objeto obtenido exitosamente: ﻿Book,Author(s),Original language,First published,Approximate sales in millions,Genre
A Tale of Two Cities,Charles Dickens,English,1859,200,Historical fiction
The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200,Novella
Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,Fantasy
And Then There Were None,Agatha Christie,English,1939,100,Mystery
Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100,Family saga
The Hobbit,J. R. R. Tolkien,English,1937,100,Fantasy
"The Lion, the Witch and the Wardrobe",C. S. Lewis,English,1950,85,"Fantasy, Children's fiction"
She: A History of Adventure,H. Rider Haggard,English,1887,83,Adventure
Vardi Wala Gunda (वर्दी वाला गुंडा),Ved Prakash Sharma,Hindi,1992,80,Detective
The Da V

## Crear S3 Estándar - Acceso poco frecuente, crear un cubo y añadir un objeto y obtener le objeto

In [30]:
# Crear un bucket S3 con la clase de almacenamiento "Acceso poco frecuente"
bucket_name_infrequent_access = 'bucket-acceso-poco-frecuente'  # Cambia el nombre para que sea único

try:
    s3_client.create_bucket(Bucket=bucket_name_infrequent_access)
    print(f"Bucket {bucket_name_infrequent_access} creado exitosamente")
except Exception as e:
    print("Error al crear el bucket S3:", str(e))

# Subir un archivo CSV con la clase de almacenamiento "Acceso poco frecuente"
file_path_infrequent_access = '/home/feliciano/SBD/Boto3/best-selling-books.csv'
file_key_infrequent_access = 'best-selling-books-infrequent-access.csv'

try:
    s3_client.upload_file(file_path_infrequent_access, bucket_name_infrequent_access, file_key_infrequent_access, ExtraArgs={'StorageClass': 'STANDARD_IA'})
    print(f"Archivo {file_path_infrequent_access} subido exitosamente a {file_key_infrequent_access} en el bucket {bucket_name_infrequent_access}")
except Exception as e:
    print("Error al subir el archivo CSV:", str(e))

# Obtener el objeto subido
try:
    response_infrequent_access = s3_client.get_object(Bucket=bucket_name_infrequent_access, Key=file_key_infrequent_access)
    print(f"Objeto obtenido exitosamente: {response_infrequent_access['Body'].read().decode('utf-8')}")
except Exception as e:
    print("Error al obtener el objeto del bucket S3:", str(e))

Bucket bucket-acceso-poco-frecuente creado exitosamente
Archivo /home/feliciano/SBD/Boto3/best-selling-books.csv subido exitosamente a best-selling-books-infrequent-access.csv en el bucket bucket-acceso-poco-frecuente
Objeto obtenido exitosamente: ﻿Book,Author(s),Original language,First published,Approximate sales in millions,Genre
A Tale of Two Cities,Charles Dickens,English,1859,200,Historical fiction
The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200,Novella
Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,Fantasy
And Then There Were None,Agatha Christie,English,1939,100,Mystery
Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100,Family saga
The Hobbit,J. R. R. Tolkien,English,1937,100,Fantasy
"The Lion, the Witch and the Wardrobe",C. S. Lewis,English,1950,85,"Fantasy, Children's fiction"
She: A History of Adventure,H. Rider Haggard,English,1887,83,Adventure
Vardi Wala Gunda (वर्दी वाला गुंडा),Ved Prakash Sharma,Hindi,1992,80,D

## Crear S3 Intelligent-Tiering, crear un cubo y añadir un objeto y obtener le objeto

In [32]:
import uuid

# Crear un bucket S3 con la clase de almacenamiento "Intelligent-Tiering"
bucket_name_intelligent_tiering = f'bucket-intelligent-tiering-{uuid.uuid4()}'  # Generar un nombre único

try:
	s3_client.create_bucket(Bucket=bucket_name_intelligent_tiering)
	print(f"Bucket {bucket_name_intelligent_tiering} creado exitosamente")
except Exception as e:
	print("Error al crear el bucket S3:", str(e))

# Subir un archivo CSV con la clase de almacenamiento "Intelligent-Tiering"
file_path_intelligent_tiering = '/home/feliciano/SBD/Boto3/best-selling-books.csv'
file_key_intelligent_tiering = 'best-selling-books-intelligent-tiering.csv'

try:
	s3_client.upload_file(file_path_intelligent_tiering, bucket_name_intelligent_tiering, file_key_intelligent_tiering, ExtraArgs={'StorageClass': 'INTELLIGENT_TIERING'})
	print(f"Archivo {file_path_intelligent_tiering} subido exitosamente a {file_key_intelligent_tiering} en el bucket {bucket_name_intelligent_tiering}")
except Exception as e:
	print("Error al subir el archivo CSV:", str(e))

# Obtener el objeto subido
try:
	response_intelligent_tiering = s3_client.get_object(Bucket=bucket_name_intelligent_tiering, Key=file_key_intelligent_tiering)
	print(f"Objeto obtenido exitosamente: {response_intelligent_tiering['Body'].read().decode('utf-8')}")
except Exception as e:
	print("Error al obtener el objeto del bucket S3:", str(e))

Bucket bucket-intelligent-tiering-706cc9bf-864f-426f-84b0-9807b91dbbb4 creado exitosamente
Archivo /home/feliciano/SBD/Boto3/best-selling-books.csv subido exitosamente a best-selling-books-intelligent-tiering.csv en el bucket bucket-intelligent-tiering-706cc9bf-864f-426f-84b0-9807b91dbbb4
Objeto obtenido exitosamente: ﻿Book,Author(s),Original language,First published,Approximate sales in millions,Genre
A Tale of Two Cities,Charles Dickens,English,1859,200,Historical fiction
The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200,Novella
Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,Fantasy
And Then There Were None,Agatha Christie,English,1939,100,Mystery
Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100,Family saga
The Hobbit,J. R. R. Tolkien,English,1937,100,Fantasy
"The Lion, the Witch and the Wardrobe",C. S. Lewis,English,1950,85,"Fantasy, Children's fiction"
She: A History of Adventure,H. Rider Haggard,English,1887,83,Adventur

## Crear S3 Glacier, crear un cubo y añadir un objeto y obtener le objeto

In [ ]:
# Nombre único del bucket Glacier
bucket_name_glacier = 'bucket-glacier'  # Cámbialo para que sea único

# Crear bucket (sin LocationConstraint para us-east-1)
try:
    s3_client.create_bucket(Bucket=bucket_name_glacier)
    print(f"Bucket {bucket_name_glacier} creado exitosamente")
except Exception as e:
    print("Error al crear el bucket S3:", str(e))

# Ruta del archivo a subir
file_path_glacier = '/home/feliciano/SBD/Boto3/best-selling-books.csv'
file_key_glacier = 'archivos-glacier/best-selling-books.csv'  # Se almacena en una "carpeta"

# Subir archivo con la clase de almacenamiento "Glacier"
try:
    s3_client.upload_file(
        file_path_glacier, 
        bucket_name_glacier, 
        file_key_glacier, 
        ExtraArgs={'StorageClass': 'GLACIER'}
    )
    print(f"Archivo {file_path_glacier} subido exitosamente en {file_key_glacier} con clase Glacier")
except Exception as e:
    print("Error al subir el archivo CSV:", str(e))

# Solicitar restauración del archivo desde Glacier (si lo necesitas)
try:
    restore_request = s3_client.restore_object(
        Bucket=bucket_name_glacier,
        Key=file_key_glacier,
        RestoreRequest={'Days': 2, 'GlacierJobParameters': {'Tier': 'Expedited'}}
    )
    print(f"Solicitud de restauración enviada para {file_key_glacier}")
except Exception as e:
    print("Error al solicitar restauración:", str(e))


Bucket bucket-glacier-felipe creado exitosamente
Archivo /home/feliciano/SBD/Boto3/best-selling-books.csv subido exitosamente en archivos-glacier/best-selling-books.csv con clase Glacier
Solicitud de restauración enviada para archivos-glacier/best-selling-books.csv
El objeto aún no está disponible para lectura. Debes esperar a que se restaure.


In [37]:
# Intentar obtener el objeto (solo funcionará cuando la restauración termine)
try:
    response_glacier = s3_client.get_object(Bucket=bucket_name_glacier, Key=file_key_glacier)
    print(f"Objeto obtenido exitosamente: {response_glacier['Body'].read().decode('utf-8')}")
except Exception as e:
    print("El objeto aún no está disponible para lectura. Debes esperar a que se restaure.")

Objeto obtenido exitosamente: ﻿Book,Author(s),Original language,First published,Approximate sales in millions,Genre
A Tale of Two Cities,Charles Dickens,English,1859,200,Historical fiction
The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200,Novella
Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,Fantasy
And Then There Were None,Agatha Christie,English,1939,100,Mystery
Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100,Family saga
The Hobbit,J. R. R. Tolkien,English,1937,100,Fantasy
"The Lion, the Witch and the Wardrobe",C. S. Lewis,English,1950,85,"Fantasy, Children's fiction"
She: A History of Adventure,H. Rider Haggard,English,1887,83,Adventure
Vardi Wala Gunda (वर्दी वाला गुंडा),Ved Prakash Sharma,Hindi,1992,80,Detective
The Da Vinci Code,Dan Brown,English,2003,80,Mystery thriller
Harry Potter and the Chamber of Secrets,J. K. Rowling,English,1998,77,Fantasy
Harry Potter and the Prisoner of Azkaban,J. K. Rowling,English,1999,65

## Crear S3 Glacier Deep Archive, crear un cubo y añadir un objeto y obtener le objeto

In [38]:
# Nombre del bucket (debe ser único)
bucket_name_glacier_deep = 'bucket-glacier-deep'  # Cambia el nombre para evitar colisiones

# Crear bucket (sin 'LocationConstraint' en 'us-east-1')
try:
    s3_client.create_bucket(Bucket=bucket_name_glacier_deep)
    print(f"Bucket {bucket_name_glacier_deep} creado exitosamente")
except Exception as e:
    print("Error al crear el bucket S3:", str(e))

# Ruta del archivo local y nombre en S3
file_path_glacier_deep = '/home/feliciano/SBD/Boto3/best-selling-books.csv'
file_key_glacier_deep = 'archivos-deep-archive/best-selling-books.csv'

# Subir archivo con la clase de almacenamiento "Glacier Deep Archive"
try:
    s3_client.upload_file(
        file_path_glacier_deep,
        bucket_name_glacier_deep,
        file_key_glacier_deep,
        ExtraArgs={'StorageClass': 'DEEP_ARCHIVE'}
    )
    print(f"Archivo {file_path_glacier_deep} subido exitosamente en {file_key_glacier_deep} con clase Glacier Deep Archive")
except Exception as e:
    print("Error al subir el archivo CSV:", str(e))

# Solicitar restauración del archivo desde Glacier Deep Archive
try:
    restore_request = s3_client.restore_object(
        Bucket=bucket_name_glacier_deep,
        Key=file_key_glacier_deep,
        RestoreRequest={'Days': 7, 'GlacierJobParameters': {'Tier': 'Standard'}}
    )
    print(f"Solicitud de restauración enviada para {file_key_glacier_deep}")
except Exception as e:
    print("Error al solicitar restauración:", str(e))

Bucket bucket-glacier-deep creado exitosamente
Archivo /home/feliciano/SBD/Boto3/best-selling-books.csv subido exitosamente en archivos-deep-archive/best-selling-books.csv con clase Glacier Deep Archive
Solicitud de restauración enviada para archivos-deep-archive/best-selling-books.csv


In [47]:
# Intentar obtener el objeto (fallará hasta que se restaure)
try:
    response_glacier_deep = s3_client.get_object(Bucket=bucket_name_glacier_deep, Key=file_key_glacier_deep)
    print(f"Objeto obtenido exitosamente: {response_glacier_deep['Body'].read().decode('utf-8')}")
except Exception as e:
    print("El objeto aún no está disponible para lectura. Debes esperar a que se restaure.")

El objeto aún no está disponible para lectura. Debes esperar a que se restaure.


## Hablitar el control de versiones de S3 mediante comandos y mostrar un ejemplo de un objeto modificado y mostrar dos versiones

In [50]:
import uuid

# Habilitar el control de versiones en el bucket S3
bucket_name_versioning = f'bucket-versionado-{uuid.uuid4()}'  # Generar un nombre único

try:
    s3_client.create_bucket(Bucket=bucket_name_versioning)
    print(f"Bucket {bucket_name_versioning} creado exitosamente")
except Exception as e:
    print("Error al crear el bucket S3:", str(e))

try:
    s3_client.put_bucket_versioning(
        Bucket=bucket_name_versioning,
        VersioningConfiguration={
            'Status': 'Enabled'
        }
    )
    print(f"Control de versiones habilitado en el bucket {bucket_name_versioning}")
except Exception as e:
    print("Error al habilitar el control de versiones:", str(e))

Bucket bucket-versionado-6af414e7-272d-4824-8e83-9bde9aed129e creado exitosamente
Control de versiones habilitado en el bucket bucket-versionado-6af414e7-272d-4824-8e83-9bde9aed129e


In [51]:
# Subir un archivo al bucket con control de versiones habilitado
file_path_versioning = '/home/feliciano/SBD/Boto3/best-selling-books.csv'
file_key_versioning = 'best-selling-books.csv'

try:
    s3_client.upload_file(file_path_versioning, bucket_name_versioning, file_key_versioning)
    print(f"Archivo {file_path_versioning} subido exitosamente a {file_key_versioning} en el bucket {bucket_name_versioning}")
except Exception as e:
    print("Error al subir el archivo CSV:", str(e))

Archivo /home/feliciano/SBD/Boto3/best-selling-books.csv subido exitosamente a best-selling-books.csv en el bucket bucket-versionado-6af414e7-272d-4824-8e83-9bde9aed129e


In [52]:
# Modificar el archivo y subirlo de nuevo para crear una nueva versión
try:
    with open(file_path_versioning, 'a') as f:
        f.write("\nNueva línea añadida para crear una nueva versión.")
    
    s3_client.upload_file(file_path_versioning, bucket_name_versioning, file_key_versioning)
    print(f"Archivo {file_path_versioning} modificado y subido nuevamente a {file_key_versioning} en el bucket {bucket_name_versioning}")
except Exception as e:
    print("Error al modificar y subir el archivo CSV:", str(e))

Archivo /home/feliciano/SBD/Boto3/best-selling-books.csv modificado y subido nuevamente a best-selling-books.csv en el bucket bucket-versionado-6af414e7-272d-4824-8e83-9bde9aed129e


In [53]:
# Listar las versiones del objeto
try:
    versions = s3_client.list_object_versions(Bucket=bucket_name_versioning, Prefix=file_key_versioning)
    for version in versions.get('Versions', []):
        print(f"Versión ID: {version['VersionId']}, Última modificación: {version['LastModified']}, Tamaño: {version['Size']}")
except Exception as e:
    print("Error al listar las versiones del objeto:", str(e))

Versión ID: tGvPoTTforjEhqVKete6cl.7o6pEGZ_S, Última modificación: 2025-03-15 11:11:19+00:00, Tamaño: 12953
Versión ID: gBgdVQ83tJqIgEvZVr8JrMGmITrcMMo0, Última modificación: 2025-03-15 11:10:59+00:00, Tamaño: 12900


## Realizar 3 consultas diferentes sobre el objeto .csv del S3 usando AWS Athena

In [103]:
import boto3

# Crear cliente de Athena
athena_client = boto3.client('athena', region_name='us-east-1')

# Nombre de la base de datos y tabla en Athena
database_name = 'default'  # Asegúrate de tener una base de datos creada
table_name = 'mi_tabla_csv'

# Ubicación del bucket S3 donde está el archivo CSV
bucket_name = 'bucket-estandar'  # Asegúrate de usar tu bucket S3
file_key = 's3://bucket-estandar/carpeta1/carpeta2/carpeta3/best-selling-books.csv'  # Ruta al archivo CSV en el bucket

# Crear la consulta para crear la tabla en Athena
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
    `Book` varchar(255),
    `Author(s)` varchar(255),
    `Original language` varchar(255),
    `First published` INT,
    `Approximate sales in millions` DOUBLE,
    `Genre` varchar(255)
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
STORED AS TEXTFILE
LOCATION 's3://{bucket_name}/{file_key}'
TBLPROPERTIES ('skip.header.line.count'='1');
"""

# Función para ejecutar la consulta de creación de tabla
def execute_query(query):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database_name},
        ResultConfiguration={'OutputLocation': 's3://bucket-estandar/athena-results/'}
    )
    return response['QueryExecutionId']

# Ejecutar la consulta
query_execution_id = execute_query(create_table_query)
print(f"Consulta de creación de tabla ejecutada con ID: {query_execution_id}")


Consulta de creación de tabla ejecutada con ID: b72091e4-1548-4b4f-92a9-cba0fde4d3d4


In [94]:
import time
import boto3

# Crear cliente de Athena
athena_client = boto3.client('athena', region_name='us-east-1')

# Nombre de la base de datos y tabla en Athena
database_name = 'default'  # Cambia a 'default' o a una base de datos existente
table_name = 'mi_tabla_csv'

# Ubicación del bucket S3 para los resultados de las consultas
output_location = 's3://bucket-estandar/athena-results/'

# Función para ejecutar una consulta en Athena
def execute_query(query):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database_name},
        ResultConfiguration={'OutputLocation': output_location}
    )
    return response['QueryExecutionId']

# Función para esperar a que la consulta termine
def wait_for_query_to_complete(query_execution_id):
    while True:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        state = response['QueryExecution']['Status']['State']
        
        if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            return state
        
        print(f"Esperando que la consulta termine... Estado actual: {state}")
        time.sleep(2)  # Esperar 2 segundos antes de verificar de nuevo

# Función para obtener los resultados de una consulta
def get_query_results(query_execution_id):
    return athena_client.get_query_results(QueryExecutionId=query_execution_id)

# Consulta 1: Contar el número de registros en el archivo CSV
query1 = f'SELECT COUNT(*) FROM {table_name};'
query_execution_id1 = execute_query(query1)

# Esperar a que la consulta termine antes de obtener resultados
if wait_for_query_to_complete(query_execution_id1) == 'SUCCEEDED':
    result1 = get_query_results(query_execution_id1)
    print("Resultado de la Consulta 1:", result1)
else:
    print("La consulta 1 falló o fue cancelada.")

# Consulta 2: Obtener los primeros 10 registros del archivo CSV
query2 = f'SELECT * FROM {table_name} LIMIT 10;'
query_execution_id2 = execute_query(query2)

if wait_for_query_to_complete(query_execution_id2) == 'SUCCEEDED':
    result2 = get_query_results(query_execution_id2)
    print("Resultado de la Consulta 2:", result2)
else:
    print("La consulta 2 falló o fue cancelada.")

# Consulta 3: Obtener el promedio de una columna numérica (reemplaza 'First published' por el nombre real de la columna)
query3 = f'SELECT AVG(`First published`) FROM {table_name};'
query_execution_id3 = execute_query(query3)

if wait_for_query_to_complete(query_execution_id3) == 'SUCCEEDED':
    result3 = get_query_results(query_execution_id3)
    print("Resultado de la Consulta 3:", result3)
else:
    print("La consulta 3 falló o fue cancelada.")


Esperando que la consulta termine... Estado actual: QUEUED
La consulta 1 falló o fue cancelada.
Esperando que la consulta termine... Estado actual: QUEUED
La consulta 2 falló o fue cancelada.


InvalidRequestException: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: Queries of this type are not supported

In [ ]:
import time

# Crear cliente de Athena
athena_client = boto3.client('athena', region_name='us-east-1')

# Nombre de la base de datos y tabla en Athena
database_name = 'default'  # Cambia a 'default' o a una base de datos existente
table_name = 'mi_tabla_csv'

# Ubicación del bucket S3 para los resultados de las consultas
output_location = 's3://bucket-estandar/athena-results/'

# Función para ejecutar una consulta en Athena
def execute_query(query):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database_name},
        ResultConfiguration={'OutputLocation': output_location}
    )
    return response['QueryExecutionId']

# Función para esperar a que la consulta termine
def wait_for_query_to_complete(query_execution_id):
    while True:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        state = response['QueryExecution']['Status']['State']
        
        if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            return state
        
        print(f"Esperando que la consulta termine... Estado actual: {state}")
        time.sleep(2)  # Esperar 2 segundos antes de verificar de nuevo

# Función para obtener los resultados de una consulta
def get_query_results(query_execution_id):
    return athena_client.get_query_results(QueryExecutionId=query_execution_id)
